In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import math
%matplotlib inline

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/matplotlib/__init__.py:1067: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# The Data

The data-set is composed of 16,800 characters written by 60 participants, the age range is between 19 to 40 years, and 90% of participants are right-hand. Each participant wrote each character (from ’alef’ to ’yeh’) ten times on two forms as shown in Fig. 7(a) & 7(b). The forms were scanned at the resolution of 300 dpi. Each block is segmented automatically using Matlab 2016a to determining the coordinates for each block. The database is partitioned into two sets: a training set (13,440 characters to 480 images per class) and a test set (3,360 characters to 120 images per class). Writers of training set and test set are exclusive. Ordering of including writers to test set are randomized to make sure that writers of test set are not from a single institution (to ensure variability of the test set).

** Setting up the Data **

In [2]:
arabicTrainImages = pd.read_csv("csvTrainImages 13440x1024.csv")
arabicTrainLabels = pd.read_csv("csvTrainLabel 13440x1.csv")
arabicTestImages = pd.read_csv("csvTestImages 3360x1024.csv")
arabicTestLabel = pd.read_csv("csvTestLabel 3360x1.csv")

In [3]:
print(arabicTrainImages.shape)
print(arabicTrainLabels.shape)
print(arabicTestImages.shape)
print(arabicTestLabel.shape)

(13439, 1024)
(13439, 1)
(3359, 1024)
(3359, 1)


In [4]:
trainImgs = arabicTrainImages.values
trainLabels = arabicTrainLabels.values 
testImgs = arabicTestImages.values
testLabels = arabicTestLabel.values

In [5]:
def shuffle_data(X,y):

    data = np.hstack((X,y))
    np.random.shuffle(data)
    X = data[:,:-1]
    y = data[:, -1]
    return X,y

In [6]:
[trainImgs, trainLabels] = shuffle_data(trainImgs,trainLabels)
[testImgs, testLabels] = shuffle_data(testImgs,testLabels)

In [7]:
X_train = trainImgs.reshape(13439, 32, 32,1).astype('uint8')
y_train = trainLabels.astype('uint8')
X_test = testImgs.reshape(3359, 32, 32,1).astype('uint8')
y_test = testLabels.astype('uint8')

In [8]:
X_train = np.transpose(X_train,(0,2,1,3))
X_test = np.transpose(X_test,(0,2,1,3))

In [9]:
def image_normalization(X):
    out = X/255
    return out

In [10]:
X_train = image_normalization(X_train)
X_test = image_normalization(X_test )

In [11]:
index = 1
#plt.imshow(X_test[index], cmap='Blues')
print(y_test[index])

13


In [12]:
index = 2
#plt.imshow(X_test[index], cmap='Blues')
print(y_test[index])

17


In [13]:
index = 3
#plt.imshow(X_test[index], cmap='Blues')
print(y_test[index])

14


In [14]:
#y_test[1:20]

In [15]:
def one_hot_encode(vec, vals=28):
    # one-hot encodes the 28 possible labels
    n = len(vec)
    out = np.zeros((n,vals))
    out[range(n), vec-1] = 1
    return out

In [16]:
y_train_one_hot = one_hot_encode(y_train)
y_test_one_hot = one_hot_encode(y_test)

In [17]:
#print(y_train_one_hot[0:2])
#print()
#print(y_test_one_hot[0:2])

In [18]:
def make_batches(X,y, num_of_batches = 10):
    #splits data into batches, assumes the data has already been shuffled
    #assumes data is organized where each row in the 0th axis is it's own sample 

    n = len(X)
    batch_size = math.floor(float(n)/float(num_of_batches))

    all_X_batches = []
    all_y_batches = []
    batch_start = 0
    batch_end = batch_size
    for i in range(num_of_batches):
        XBatch = X[batch_start: batch_end]
        all_X_batches.append(XBatch)

        yBatch = y[batch_start: batch_end]
        all_y_batches.append(yBatch)
        batch_start = batch_end
        batch_end += batch_size


    return all_X_batches, all_y_batches

In [19]:
class BatchFeed:
    
    def __init__(self, X_train, y_train, X_test, y_test):
        self.i = 0
        
        self.all_X_batches = []
        self.all_y_batches = []
        self.test_batch = [X_test, y_test]
        
        self.train_images = X_train
        self.train_labels = y_train
        
        self.num_of_batches = 0

        
    def set_up_batches(self, num_of_batches= 10):
        
        self.num_of_batches = num_of_batches
        
        self.all_X_batches, self.all_y_batches = make_batches(
            self.train_images, self.train_labels, 
            num_of_batches = num_of_batches)
        
    def next_batch(self):
        X = self.all_X_batches[self.i]
        y = self.all_y_batches[self.i]
        self.i += 1
        if (self.num_of_batches == self.i):
            self.i = 0
            
        return X, y

In [20]:
bf = BatchFeed(X_train, y_train_one_hot, X_test, y_test_one_hot)
bf.set_up_batches()

# Creating the Model

In [21]:
x = tf.placeholder(tf.float32, shape=[None,32,32,1])## dimensions fix
y_true = tf.placeholder(tf.float32, shape=[None, 28])

In [22]:
hold_prob = tf.placeholder(tf.float32)

In [23]:
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(init_random_dist)

def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides =[1,1,1,1], padding ='SAME')

def max_pool_2by2(x):
    return tf.nn.max_pool(x, ksize=[1,2,2,1], 
                      strides=[1,2,2,1], padding = 'SAME')

def convolutional_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x, W + b))

def normal_full_layer(input_layer, size):
        input_size = int(input_layer.get_shape()[1])
        W = init_weights([input_size, size])
        b = init_bias([size])
        return tf.matmul(input_layer, W) + b

In [24]:
convo_1 = convolutional_layer(x, shape = [4,4,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [25]:
convo_2 = convolutional_layer(convo_1_pooling, shape= [4,4,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [26]:
8*8*64

4096

In [27]:
convo_2_flat = tf.reshape(convo_2_pooling, [-1,8*8*64])

In [28]:
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

In [29]:
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob = hold_prob)

In [30]:
y_pred = normal_full_layer(full_one_dropout,28)

#### Loss Function 

In [31]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y_true, logits = y_pred))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



#### Optimizer

In [32]:
optimizer = tf.train.AdamOptimizer(learning_rate = 0.001)
train = optimizer.minimize(cross_entropy)

In [33]:
init = tf.global_variables_initializer()

## Save Model Setup

In [34]:
saver = tf.train.Saver()

## Graph Session

In [35]:
import timeit

start = timeit.default_timer()

In [36]:
#NUM_THREADS = 1
with tf.Session() as sess:
    
    #sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=NUM_THREADS))
    sess.run(tf.global_variables_initializer())
    
    for i in range(10000):
        batch = bf.next_batch()
        sess.run(train, feed_dict = {x:batch[0], y_true: batch[1], hold_prob: 0.5})
        
        # print out stats every 100 steps 
        if i%5000 == 0:

            print('Currently on step {}'.format(i))
            print('Accuracy is:')
            # test the train model
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

            acc = tf.reduce_mean(tf.cast(matches, tf.float32))

            print(sess.run(acc, feed_dict = {
                x:bf.test_batch[0], y_true: bf.test_batch[1], hold_prob:1.0}))
            print('\n')
            
    print('Currently on last step')
    print('Accuracy is:')
    # test the train model
    matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))

    acc = tf.reduce_mean(tf.cast(matches, tf.float32))

    print(sess.run(acc, feed_dict = {
        x:bf.test_batch[0], y_true: bf.test_batch[1], hold_prob:1.0}))
    print('\n')
        
    saver.save(sess,'./Arabic CNN.ckpt')

Currently on step 0
Accuracy is:
0.06370944


Currently on step 5000
Accuracy is:
0.8901459


Currently on last step
Accuracy is:
0.9336112




In [37]:
stop = timeit.default_timer()

print( stop - start )

1589.203708463
